<a href="https://colab.research.google.com/github/Marcusleeleelee/FTEC4998-4999/blob/main/FTEC4998_4999.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
# Import necessary packages
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from google.colab import drive
from tqdm import tqdm
import inspect
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Step 1: Utils - ok
def uni_list(input): return list(set(input))

In [5]:
class Dataset():
    def __init__(self, file_path):
        self.dataset = pd.read_feather(file_path)
        self.X_train, self.y_train = None, None
        self.X_test, self.y_test = None, None
        self.scalers = None
        self.pca = None
        self.label = 'loan_condition_cat'
        self.original_columns = None

    def show(self, rows=10):
        return self.dataset.head(rows)

    def basic_processing(self):
        temp_func_1 = lambda x: '<=2009' if str(x) in ['2007', '2008', '2009'] else ("[2010, 2012]" if str(x) in ['2010', '2011', '2012'] else '>=2013')
        columns_to_delete = [
            'id', 'issue_d', 'home_ownership_cat', 'income_category', 'income_cat', 'term_cat', 'application_type_cat',
            'purpose_cat', 'interest_payment_cat', 'loan_condition'
        ]
        self.dataset.drop(columns=columns_to_delete, inplace=True)
        self.dataset['grade'] = self.dataset['grade'].apply(temp_func_1)
        self.dataset['final_d'] = self.dataset['final_d'].apply(lambda x: str(x)[-4:]).apply(temp_func_1)
        self.dataset = pd.get_dummies(self.dataset, columns=['year', 'final_d', 'home_ownership', 'term', 'application_type',
                                                             'purpose', 'interest_payments', 'grade', 'region'], dtype=int)

    def train_test_split(self, test_size=0.2, random_state=42):
        X = self.dataset.drop(columns=[self.label])
        y = self.dataset[self.label]
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
        self.original_columns = X.columns

    def preprocessing_train(self):
        scaler = StandardScaler()
        self.X_train = pd.DataFrame(scaler.fit_transform(self.X_train), columns=self.original_columns)
        self.scalers = scaler

        # Perform PCA
        self.pca = PCA(n_components=30)
        pca_components = self.pca.fit_transform(self.X_train)
        self.X_train = pd.DataFrame(pca_components, columns=self.original_columns[:pca_components.shape[1]])

    def preprocessing_test(self):
        # Apply stored scalers
        self.X_test = pd.DataFrame(self.scalers.transform(self.X_test), columns=self.original_columns)

        # Apply PCA
        pca_components = self.pca.transform(self.X_test)
        self.X_test = pd.DataFrame(pca_components, columns=self.original_columns[:pca_components.shape[1]])

In [6]:
# Calculating # ok
data = Dataset('/content/drive/My Drive/Colab Notebooks/FTEC4998_9/loan_final313_processed.feather')
data.basic_processing()
data.train_test_split()
data.preprocessing_train()
data.preprocessing_test()

In [7]:
# Data conversion # ok
train_x, train_y = data.X_train, data.y_train
test_x, test_y = data.X_test, data.y_test
counts = np.mean(train_y == 1) * 100
print(counts)
print(train_x.shape, train_y.shape)
print(test_x.shape, test_y.shape)
# Ensure y_train is binary
assert set(train_y).issubset({0, 1}), "Target values must be 0 or 1 for binary classification."
# Move to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

7.5910370853482805
(709903, 30) (709903,)
(177476, 30) (177476,)


In [26]:
# Train, predict, and accuracy functions
def train_model(model): # ok
    print("Training.")
    model.train()
    for epoch in range(model.epochs):
        model.optimizer.zero_grad()
        outputs = model(model.train_x)
        loss = model.criterion(outputs, model.train_y)
        loss.backward()
        model.optimizer.step()
        if epoch % 5 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

def predict_model(model, X):
    print('Predicting.')
    model.eval()
    with torch.no_grad():
        outputs = model(X).squeeze()
        return (outputs > 0.5).float()

def calculate_accuracy(model, X, y):
    print('Calculating Accuracy.')
    X = X.to(next(model.parameters()).device)
    y = y.to(next(model.parameters()).device)
    predictions = predict_model(model, X)

    # Ensure predictions and labels are the same shape
    predictions = predictions.squeeze()
    y = y.squeeze()

    correct = (predictions == y).sum().item()
    accuracy = correct / y.size(0)
    return accuracy

def df_to_tensor(x, y):
    assert isinstance(x, pd.DataFrame) and isinstance(y, pd.Series)
    return torch.tensor(x.to_numpy(), dtype=torch.float32).to(device), torch.tensor(y.values.ravel(), dtype=torch.float32).unsqueeze(1).to(device)
test_x, test_y = data.X_test, data.y_test
test_x_tensor, test_y_tensor = df_to_tensor(data.X_test, data.y_test)

In [59]:
# MLP model
class ANN(nn.Module):
    def __init__(self, train_x, train_y, lr=0.001):
        super(ANN, self).__init__()
        self.train_y = torch.tensor(train_y.values.ravel(), dtype=torch.float32).unsqueeze(1).to(device)
        self.train_x = torch.tensor(train_x.to_numpy(), dtype=torch.float32).to(device)
        self.input_dim = self.train_x.shape[1]
        self.net = nn.Sequential(
            nn.Linear(self.input_dim, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.LeakyReLU(0.03),
            nn.Dropout(0.3),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
        self._initialize_weights()
        self.criterion = nn.BCELoss()
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.epochs = 10

    def forward(self, x):
        return self.net(x)
    def _initialize_weights(self):
        for m in self.net:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

# Logistic Regression as a neural network
class LogisticRegressionModel(nn.Module):
    def __init__(self, train_x, train_y, lr=0.01):
        super(LogisticRegressionModel, self).__init__()
        self.train_y = torch.tensor(train_y.values.ravel(), dtype=torch.float32).unsqueeze(1).to(device)
        self.train_x = torch.tensor(train_x.to_numpy(), dtype=torch.float32).to(device)
        self.input_dim = self.train_x.shape[1]
        self.net = nn.Sequential(
            nn.Linear(self.input_dim, 1),
            nn.Sigmoid()
        )
        self.criterion = nn.BCELoss()
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.epochs = 700

    def forward(self, x):
        return self.net(x)

# SVM
class SVMClassifier():
    def __init__(self, train_x, train_y, fraction=0.1, n_samples=10000000000000):
        # Sample a fraction of the data
        n = min(int(len(train_x) * fraction), n_samples)
        self.train_x = train_x.iloc[:n, :]
        self.train_y = train_y.iloc[:n]
        self.model = None
    def fit(self):
        print("Training.")
        # Use Bagging with SVM
        self.model = BaggingClassifier(
            estimator=SVC(C=0.1, kernel='poly', degree=5, gamma='scale'),
            n_estimators=6,
            random_state=42,
            max_samples= 0.05
        )
        self.model.fit(self.train_x, self.train_y)

    def predict(self, X):
        print('Predicting.')
        return self.model.predict(X).astype(float)

    def calculate_accuracy(self, X, y):
        print('Calculating Accuracy.')
        predictions = self.predict(X)
        accuracy = accuracy_score(y, predictions)
        return accuracy

# NB
class NaiveBayesClassifier():
    def __init__(self, train_x, train_y, priors=None, var_smoothing=1e-9):
        self.model = GaussianNB(priors=priors, var_smoothing=var_smoothing)
        self.train_x = train_x
        self.train_y = train_y

    def fit(self):
        self.model.fit(self.train_x, self.train_y)

    def predict(self, X):
        return self.model.predict(X).astype(float)

    def calculate_accuracy(self, X, y):
        predictions = self.predict(X)
        accuracy = accuracy_score(y, predictions)
        return accuracy
# RF
class RandomForestModel():
    def __init__(self, train_x, train_y, n_estimators=10, max_depth=None, random_state=42):
        self.model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state, max_samples = 0.05)
        self.train_x = train_x
        self.train_y = train_y

    def fit(self):
        self.model.fit(self.train_x, self.train_y)

    def predict(self, X):
        return self.model.predict(X).astype(float)

    def calculate_accuracy(self, X, y):
        predictions = self.predict(X)
        accuracy = accuracy_score(y, predictions)
        return accuracy

In [60]:
class Adaboost():
    def __init__(self, classes_dict, train_x, train_y):
        self.classes_dict = classes_dict
        self.model_order = list(classes_dict.keys())
        self.train_x = train_x
        self.train_y = train_y
        self.trained_model = {}
    def data_resampling(self, current_prediction): pass
    def training(self):
        for i in self.model_order:
            print("*" * 100)
            print(f'Training --- {i}')
            current_model = self.classes_dict[i](self.train_x, self.train_y)
            methods = inspect.getmembers(current_model, predicate=inspect.ismethod)
            if 'fit' in [i for i, j in methods]:
                current_model.fit()
                current_prediction = current_model.predict(current_model.train_x)
                train_accuracy = current_model.calculate_accuracy(current_model.train_x, current_model.train_y)
            else:
                current_model.to(device)
                train_model(current_model)
                current_prediction = predict_model(current_model, current_model.train_x).cpu().numpy()
                train_accuracy = calculate_accuracy(current_model, current_model.train_x, current_model.train_y)
            print(f'{i} training accuracy:', train_accuracy)
            print("-------->", current_prediction)
            print("-------->", type(current_prediction))
            print(np.unique("-------->", current_prediction))
            self.trained_model.update({i: current_model})
            self.data_resampling(current_prediction)

adModel = Adaboost(classes_dict={"ANN": ANN, "LR": LogisticRegressionModel, "SVM": SVMClassifier, "NB": NaiveBayesClassifier, "RF": RandomForestModel}, train_x=train_x, train_y=train_y)
adModel.training()


****************************************************************************************************
Training --- ANN
Training.
Epoch 0, Loss: 0.9193267226219177
Epoch 5, Loss: 0.8005325198173523
Predicting.
Calculating Accuracy.
Predicting.
ANN training accuracy: 0.8232181016279689
[1. 0. 0. ... 0. 0. 0.]
<class 'numpy.ndarray'>
[0. 1.]
****************************************************************************************************
Training --- LR
Training.
Epoch 0, Loss: 0.6851571798324585
Epoch 5, Loss: 0.63309645652771
Epoch 10, Loss: 0.5961793661117554
Epoch 15, Loss: 0.5686639547348022
Epoch 20, Loss: 0.545014500617981
Epoch 25, Loss: 0.5195611715316772
Epoch 30, Loss: 0.4984554350376129
Epoch 35, Loss: 0.47974124550819397
Epoch 40, Loss: 0.4626387655735016
Epoch 45, Loss: 0.4467485249042511
Epoch 50, Loss: 0.4320027828216553
Epoch 55, Loss: 0.4218024015426636
Epoch 60, Loss: 0.40886273980140686
Epoch 65, Loss: 0.39686763286590576
Epoch 70, Loss: 0.38571488857269287
Epoch 75,

In [61]:
print(adModel.model_order)

['ANN', 'LR', 'SVM', 'NB', 'RF']
